In [1]:
import pandas as pd
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [2]:
# Load the datasets
train_data = pd.read_csv('tweets-ext.csv')
test_data = pd.read_csv('tweets-test.csv')
val_data = pd.read_csv('tweets-valid.csv')

In [3]:
# Preprocess the data
X_train = train_data['tweet']
y_train = train_data['label']
X_test = test_data['tweet']
y_test = test_data['label']
X_val = val_data['tweet']
y_val = val_data['label']

In [4]:
# Tokenize the text data
tokenizer = Tokenizer()
tokenizer.fit_on_texts(X_train)
X_train_seq = tokenizer.texts_to_sequences(X_train)
X_test_seq = tokenizer.texts_to_sequences(X_test)
X_val_seq = tokenizer.texts_to_sequences(X_val)

In [5]:
# Pad sequences to a fixed length
max_length = 280  # maximum length of a tweet
X_train_pad = pad_sequences(X_train_seq, maxlen=max_length, padding='post')
X_test_pad = pad_sequences(X_test_seq, maxlen=max_length, padding='post')
X_val_pad = pad_sequences(X_val_seq, maxlen=max_length, padding='post')

In [6]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(input_dim=len(tokenizer.word_index) + 1, output_dim=100, input_length=max_length),
    tf.keras.layers.Bidirectional(tf.keras.layers.GRU(128, return_sequences=True)),
    tf.keras.layers.Bidirectional(tf.keras.layers.GRU(64, return_sequences=True)),
    tf.keras.layers.Bidirectional(tf.keras.layers.GRU(32)),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(1, activation='sigmoid')
])


In [7]:
# Compile the model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [8]:
# Train the model
model.fit(X_train_pad, y_train, epochs=10, batch_size=32, validation_data=(X_val_pad, y_val))

Epoch 1/10
458/458 [==============================] - 305s 646ms/step - loss: -262.7709 - accuracy: 0.3887 - val_loss: -2698.2651 - val_accuracy: 0.4673
Epoch 2/10
458/458 [==============================] - 255s 557ms/step - loss: -5037.4077 - accuracy: 0.4305 - val_loss: -70.9851 - val_accuracy: 0.4147
Epoch 3/10
458/458 [==============================] - 561s 1s/step - loss: -824.5348 - accuracy: 0.3878 - val_loss: -2255.9189 - val_accuracy: 0.3333
Epoch 4/10
458/458 [==============================] - 574s 1s/step - loss: -3335.8699 - accuracy: 0.3728 - val_loss: -6536.2339 - val_accuracy: 0.3333
Epoch 5/10
458/458 [==============================] - 694s 2s/step - loss: -7775.8462 - accuracy: 0.3695 - val_loss: -13987.1035 - val_accuracy: 0.3333
Epoch 6/10
458/458 [==============================] - 753s 2s/step - loss: -13046.5488 - accuracy: 0.3403 - val_loss: -24666.3887 - val_accuracy: 0.3333
Epoch 7/10
458/458 [==============================] - 569s 1s/step - loss: -21460.6035 - 

In [9]:
# Evaluate the model on test data
loss, accuracy = model.evaluate(X_test_pad, y_test)
print('Test Loss:', loss)
print('Test Accuracy:', accuracy)

71/71 [==============================] - 35s 494ms/step - loss: -98492.8984 - accuracy: 0.3333
Test Loss: -98492.8984375
Test Accuracy: 0.3333333432674408
